In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Reading Train and Test data sets from CSV file

In [ ]:
# Uncomment this code when you are running from kaggle
# train_data = pd.read_csv('/kaggle/input/titanic/train.csv')
# test_data = pd.read_csv('/kaggle/input/titanic/test.csv')
# gs = pd.read_csv('/kaggle/input/titanic/gender_submission.csv')
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
gs = pd.read_csv('gender_submission.csv')

### Printing Values of Train and Test data

In [ ]:
train_data.head()

In [ ]:
test_data.head()

### Checking Empty and Null cells in Columns

In [ ]:
train_data.isnull().sum()

In [ ]:
test_data.isnull().sum()

### Dropping some unused columns from dataframes

In [ ]:
train_data = train_data.drop(columns=['Name','Ticket','Fare','Cabin','PassengerId','Age'])
test_data = test_data.drop(columns=['Name','Ticket','Fare','Cabin','PassengerId','Age'])

### Replacing Missing values in for Embarked columns

In [ ]:
train_data.Embarked = train_data.Embarked.fillna(train_data['Embarked'].mode()[0])

### Converting dataframes to numpy arrays

In [ ]:
X_train_data = train_data[['Pclass','Sex','SibSp','Parch','Embarked']].to_numpy()
y_train_Data = train_data['Survived'].to_numpy()
X_test_data = test_data[['Pclass','Sex','SibSp','Parch','Embarked']].to_numpy()
y_test_data = gs['Survived'].to_numpy()

### Feature Encoding using OneHotEncoder - Train and Test data sets

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[1])],remainder='passthrough')
X_train_data = np.array(ct.fit_transform(X_train_data))

ct = ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[-1])],remainder='passthrough')
X_train_data = np.array(ct.fit_transform(X_train_data))

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[1])],remainder='passthrough')
X_test_data = np.array(ct.fit_transform(X_test_data))

ct = ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[-1])],remainder='passthrough')
X_test_data = np.array(ct.fit_transform(X_test_data))

### Applying Standard scaling

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train_data[1:2,5:7] = sc.fit_transform(X_train_data[1:2,5:7])
X_test_data[1:2,5:7] = sc.transform(X_test_data[1:2,5:7])

### Logistic Regression - Confusion Matrix and Accuracy score

In [ ]:
from sklearn.linear_model import LogisticRegression
regressor = LogisticRegression(max_iter=300)
regressor.fit(X_train_data,y_train_Data)

In [ ]:
y_pred = regressor.predict(X_test_data)
from sklearn.metrics import confusion_matrix,accuracy_score
logistic_confusionMatrix = confusion_matrix(y_pred,y_test_data)
logistic_Accuracy = accuracy_score(y_pred,y_test_data)

### KNN Classifier - Confusion Matrix and Accuracy score

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knnclassifier = KNeighborsClassifier(n_neighbors=5)
knnclassifier.fit(X_train_data,y_train_Data)

In [ ]:
y_pred_knn = knnclassifier.predict(X_test_data)
knn_confusionMatrix = confusion_matrix(y_pred_knn,y_test_data)
knn_Accuracy = accuracy_score(y_pred_knn,y_test_data)

### Support Vector Classifier - Confusion Matrix and Accuracy score

In [ ]:
from sklearn.svm import SVR
svr = SVR(kernel='rbf')
svr.fit(X_train_data,y_train_Data)

In [ ]:
y_pred_svr = knnclassifier.predict(X_test_data)
svr_confusionMatrix = confusion_matrix(y_pred_svr,y_test_data)
svr_Accuracy = accuracy_score(y_pred_svr,y_test_data)

### Random Forest Classifier - Confusion Matrix and Accuracy score

In [ ]:
from sklearn.ensemble import RandomForestClassifier
randomregressor = RandomForestClassifier(n_estimators=200,criterion='entropy',max_depth=5, random_state=1)
randomregressor.fit(X_train_data,y_train_Data)

In [ ]:
y_pred_rand = randomregressor.predict(X_test_data)
rand_confusionMatrix = confusion_matrix(y_pred_rand,y_test_data)
rand_Accuracy = accuracy_score(y_pred_rand,y_test_data)

### Confusion Matrix and Accuracy Score for Each Algorithms

In [ ]:
numpy_data = np.array([[logistic_confusionMatrix,logistic_Accuracy],[knn_confusionMatrix,knn_Accuracy],[svr_confusionMatrix,svr_Accuracy],[rand_confusionMatrix,rand_Accuracy]],dtype=object)
df = pd.DataFrame(data=numpy_data, index=["Logistic Regression", "KNN Classifier","Support Vector Classifier","Random Forest Classifier"], columns=["Confusion Matrix","Accuracy Score"])
print(df.sort_values(by=["Accuracy Score"]))

## Final Output

In [ ]:
output = pd.DataFrame({'PassengerId': gs.PassengerId, 'Survived': y_pred_rand})
output.to_csv('my_submission.csv', index=False)
print("Your submission was successfully saved!")